## Loading Packages

In [2]:
import numpy as np
import pandas as pd
import re
import warnings

import seaborn as sn
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

## Loading Datasets

In [3]:
df = pd.read_csv("Datasets/games_details.csv")
seasons = pd.read_csv("Datasets/games.csv")[["GAME_ID", "SEASON"]]

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Data Overview

In [4]:
df.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,22100213,1610612764,WAS,Washington,203484,Kentavious Caldwell-Pope,Kentavious,F,NaN,27:41,1.0,6.0,0.167,0.0,5.0,0.000,1.0,1.0,1.0,1.0,5.0,6.0,2.0,1.0,0.0,1.0,0.0,3.0,2.0
1,22100213,1610612764,WAS,Washington,1628398,Kyle Kuzma,Kyle,F,NaN,30:28,2.0,12.0,0.167,1.0,8.0,0.125,0.0,0.0,0.0,1.0,4.0,5.0,3.0,1.0,2.0,1.0,1.0,5.0,-14.0
2,22100213,1610612764,WAS,Washington,1629655,Daniel Gafford,Daniel,C,NaN,24:21,9.0,12.0,0.750,0.0,0.0,0.000,2.0,5.0,0.4,2.0,7.0,9.0,1.0,2.0,1.0,1.0,4.0,20.0,-2.0
3,22100213,1610612764,WAS,Washington,203078,Bradley Beal,Bradley,G,NaN,35:07,9.0,20.0,0.450,5.0,11.0,0.455,1.0,1.0,1.0,0.0,3.0,3.0,7.0,2.0,0.0,2.0,3.0,24.0,-9.0
4,22100213,1610612764,WAS,Washington,203915,Spencer Dinwiddie,Spencer,G,NaN,28:34,0.0,5.0,0.000,0.0,4.0,0.000,0.0,0.0,0.0,0.0,3.0,3.0,2.0,0.0,0.0,2.0,1.0,0.0,-5.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626111 entries, 0 to 626110
Data columns (total 29 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   GAME_ID            626111 non-null  int64  
 1   TEAM_ID            626111 non-null  int64  
 2   TEAM_ABBREVIATION  626111 non-null  object 
 3   TEAM_CITY          626111 non-null  object 
 4   PLAYER_ID          626111 non-null  int64  
 5   PLAYER_NAME        626111 non-null  object 
 6   NICKNAME           10485 non-null   object 
 7   START_POSITION     239375 non-null  object 
 8   COMMENT            102359 non-null  object 
 9   MIN                523751 non-null  object 
 10  FGM                523751 non-null  float64
 11  FGA                523751 non-null  float64
 12  FG_PCT             523751 non-null  float64
 13  FG3M               523751 non-null  float64
 14  FG3A               523751 non-null  float64
 15  FG3_PCT            523751 non-null  float64
 16  FT

## Data Cleaning


removing the game_id,team_id, player_ids from the stats

In [6]:
df.drop(["GAME_ID", "TEAM_ID", "PLAYER_ID"], axis=1).describe().transpose()

,count,mean,std,min,25%,50%,75%,max
FGM,523751.0,3.574498,3.019116,0.0,1.00,3.000,5.000,28.0
FGA,523751.0,7.878062,5.657840,0.0,3.00,7.000,11.000,50.0
FG_PCT,523751.0,0.415843,0.251636,0.0,0.25,0.429,0.571,1.0
FG3M,523751.0,0.753051,1.205652,0.0,0.00,0.000,1.000,14.0
FG3A,523751.0,2.115030,2.521680,0.0,0.00,1.000,3.000,24.0
FG3_PCT,523751.0,0.197427,0.289293,0.0,0.00,0.000,0.375,1.0
FTM,523751.0,1.738716,2.356272,0.0,0.00,1.000,3.000,26.0
FTA,523751.0,2.294722,2.892114,0.0,0.00,2.000,4.000,39.0
FT_PCT,523751.0,0.436454,0.427786,0.0,0.00,0.500,0.909,1.0
OREB,523751.0,1.028187,1.401890,0.0,0.00,1.000,2.000,18.0


Removing the duplicated data from the dataset including players that might appear more that once

In [7]:
df[df.duplicated(subset=["GAME_ID", "PLAYER_ID"], keep="first")]

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
23427,22000069,1610612763,MEM,Memphis,1628960,Grayson Allen,NaN,NaN,DNP - Injury/Illness,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23428,22000069,1610612763,MEM,Memphis,1629007,Jontay Porter,NaN,NaN,DNP - Injury/Illness,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23429,22000069,1610612763,MEM,Memphis,1630214,Xavier Tillman,NaN,NaN,DNP - Injury/Illness,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23430,22000069,1610612766,CHA,Charlotte,1630176,Vernon Carey Jr.,NaN,NaN,DNP - Coach's Decision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23431,22000069,1610612766,CHA,Charlotte,1628370,Malik Monk,NaN,NaN,DNP - Coach's Decision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40312,22000002,1610612747,LAL,Los Angeles,2544,LeBron James,NaN,F,NaN,28:21,7.0,17.0,0.412,3.0,8.0,0.375,5.0,7.0,0.714,2.0,3.0,5.0,5.0,0.0,0.0,4.0,2.0,22.0,2.0
40313,22000002,1610612747,LAL,Los Angeles,203484,Kentavious Caldwell-Pope,NaN,G,NaN,23:03,2.0,4.0,0.500,1.0,3.0,0.333,4.0,4.0,1.000,0.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,9.0,-9.0
40314,22000002,1610612747,LAL,Los Angeles,203471,Dennis Schroder,NaN,G,NaN,27:48,5.0,15.0,0.333,2.0,5.0,0.400,2.0,4.0,0.500,1.0,11.0,12.0,8.0,0.0,0.0,1.0,4.0,14.0,-10.0
40315,22000002,1610612747,LAL,Los Angeles,1628398,Kyle Kuzma,NaN,NaN,NaN,26:38,6.0,11.0,0.545,1.0,3.0,0.333,2.0,2.0,1.000,0.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,15.0,1.0


In [9]:
df.drop_duplicates(subset=["GAME_ID", "PLAYER_ID"], keep="first", inplace=True)

## Data Filling

We want to group up and sum the statistics for every player within each unique game played. We expect some columns to lose value by doing so especially player_id, the plus minus  and the shot percentage lose meaning. However some of these can be recalculated. This will remove some NaN value and some columns not really that important

In [10]:
df = df.groupby(["GAME_ID", "TEAM_ID"]).sum().reset_index().drop(['PLAYER_ID', 'FG_PCT','FG3_PCT','FT_PCT','PLUS_MINUS'], axis= 1)

In [14]:
df["FG_PCT"] = df["FGM"]/df["FGA"]*100
df["FG3_PCT"] = df["FG3M"]/df["FG3A"]*100
df["FT_PCT"] = df["FTM"]/df["FTA"]*100

Removing anomalous data

In [23]:
print (f"Dataset has {df.isna().sum().sum()} null values.\n")
print ("There seems to a game with 3 unique rows.\n")

print(df["GAME_ID"].value_counts().sort_values().tail())

display(df[df["GAME_ID"] == 10500109])

#dropping this row due for data readability
df = df.drop(index=335, axis=0).reset_index().drop("index", axis=1)

Dataset has 3 null values.

There seems to a game with 3 unique rows.

20701124    2
21100481    2
21200784    2
21000146    2
10500109    3
Name: GAME_ID, dtype: int64


,GAME_ID,TEAM_ID,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,FG_PCT,FG3_PCT,FT_PCT
334,10500109,1610612742,36.0,73.0,5.0,16.0,26.0,35.0,10.0,27.0,37.0,18.0,8.0,9.0,13.0,19.0,103.0,49.315068,31.250000,74.285714
335,10500109,1610612756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
336,10500109,1610612765,40.0,87.0,4.0,9.0,13.0,15.0,15.0,26.0,41.0,20.0,7.0,11.0,13.0,29.0,97.0,45.977011,44.444444,86.666667


We need to develop a "victory" label to develop our ML model. Also need to factor in miss data entire due to instances where the games tie and have to go into overtime.

The "victory" labels is calculated by comparing the point made in a unique game and defining it as a  "Yes" or "No"

In [ ]:
# sort the datafrome by GAME_Id to avoid in the following code to define the victory label
df = df.sort_values("GAME_ID")
df["VICTORY"] = ""

#check for victory condition and update the "Victory" column
for i in range(0, len(df)-1, 2):
    if df["PTS"][i] > df["PTS"][i+1]:
        df.loc[i, "VICTORY"] = "YES"
        df.loc[i+1, "VICTORY"] = "NO"
    elif df["PTS"][i] < df["PTS"][i+1]:
        df.loc[i, "VICTORY"] = "NO"
        df.loc[i+1, "VICTORY"] = "YES"
    else:
        print("Potential Tie game")
        

## Exploratory Data Analysis


## Hypothesis Builiding

## Feature Engineering & Selection


## ML Model Selection

## ML Fit & Training

## ML Validation Testing

## ML Prediction and Analysis